#### Libraries importeren

In [22]:
import sys
assert sys.version_info >= (3, 5)
import sklearn
assert sklearn.__version__ >= "0.20"
import numpy as np
import pandas as pd
import os
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#### Data ophalen

In [23]:
df=pd.read_csv("../data/afsprakenn.csv", sep=';')
df.head()

C:\Users\anna2\AppData\Local\Temp\ipykernel_20892\523483477.py:1: DtypeWarning: Columns (23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("../data/afsprakenn.csv", sep=';')


,afspraak_id,afspraak_geannuleerd,afspraak_beginuur,afspraak_leadTime,afspraak_duur,afspraak_noShow,afspraak_aantalHerplanningen,afspraak_beschrijving,afspraak_specialisme,afspraak_arts,patient_aantalEerdereAfspraken,patient_dagenSindsVorigeAfspraak,patient_aantalAnnulaties,patient_aantalLaattijdigAnnulaties,patient_aantalAfsprakenDieDag,patient_stiptheid,patient_aantalNoShows,afspraak_postcode,afspraak_weekdag,afspraak_maand,afspraak_isFeestdag,afspraak_isSchoolvakantie,patient_leeftijd,patient_geslacht,patient_dubieuzeBetaler,patient_burgerlijkeStaat,patient_postcode,patient_nieuw,afstand
0,39564122,1,8,42,60,1,0,214,10,18932,0,NaN,NaN,NaN,2,NaN,NaN,9000.0,5,6,0,0,122.0,1,0.0,0,ZZNF,0,NaN
1,39564123,1,9,42,20,1,0,215,10,6310,1,0.0,1.0,0.0,2,NaN,1.0,9000.0,5,6,0,0,122.0,1,0.0,0,ZZNF,0,NaN
2,40387836,0,16,27,15,1,0,266,10,7692,0,NaN,NaN,NaN,1,NaN,NaN,9000.0,1,6,0,0,22.0,1,0.0,1,9030,0,0.048794
3,50859232,0,10,93,15,0,0,266,114,7692,1,430.0,0.0,0.0,1,NaN,1.0,9000.0,4,8,0,1,23.0,1,0.0,1,9030,0,0.048794
4,31478199,0,9,10,15,0,0,1141,126,18924,0,NaN,NaN,NaN,1,NaN,NaN,9000.0,1,11,0,0,21.0,1,0.0,5,2220,0,1.003979


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1988510 entries, 0 to 1988509
Data columns (total 29 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   afspraak_id                         int64  
 1   afspraak_geannuleerd                int64  
 2   afspraak_beginuur                   int64  
 3   afspraak_leadTime                   int64  
 4   afspraak_duur                       int64  
 5   afspraak_noShow                     int64  
 6   afspraak_aantalHerplanningen        int64  
 7   afspraak_beschrijving               int64  
 8   afspraak_specialisme                int64  
 9   afspraak_arts                       int64  
 10  patient_aantalEerdereAfspraken      int64  
 11  patient_dagenSindsVorigeAfspraak    float64
 12  patient_aantalAnnulaties            float64
 13  patient_aantalLaattijdigAnnulaties  float64
 14  patient_aantalAfsprakenDieDag       int64  
 15  patient_stiptheid                   float64
 16  

#### Data cleanen

In [25]:
# geannuleerde afspraken verwijderen (0= niet geannuleerd, 1 = geannuleerd, 2 ="laattijdig" geannuleerd)
df = df.drop(df[df['afspraak_geannuleerd'] == 1].index)
df = df.drop(df[df['afspraak_geannuleerd'] == 2].index)

df = df.drop(columns=['afspraak_geannuleerd'])

In [26]:
df = df.drop(['afspraak_postcode', 'patient_postcode', 'afspraak_isFeestdag'], axis=1)

##### Lege velden invullen

In [27]:
df['patient_aantalNoShows'] = df['patient_aantalNoShows'].fillna(0)

In [28]:
# df['patient_eersteBetaler_categorie'] = df['patient_eersteBetaler_categorie'].fillna('N')
# df['patient_eersteBetaler_categorie'].value_counts()

In [29]:
# df['patient_tweedeBetaler_categorie'] = df['patient_tweedeBetaler_categorie'].fillna('N')
# df['patient_tweedeBetaler_categorie'].value_counts()

In [31]:
df['patient_geslacht'].value_counts()

1    994057
0    724893
2         4
Name: patient_geslacht, dtype: int64

In [32]:
df['patient_stiptheid'] = df['patient_stiptheid'].fillna(df['patient_stiptheid'].median())

In [33]:
df['afstand'] = df['afstand'].fillna(df['afstand'].mean())

In [34]:
df['patient_aantalLaattijdigAnnulaties'] = df['patient_aantalLaattijdigAnnulaties'].fillna(0)

In [35]:
df['patient_aantalAnnulaties'] = df['patient_aantalAnnulaties'].fillna(0)

In [36]:
df['patient_dagenSindsVorigeAfspraak'] = df['patient_dagenSindsVorigeAfspraak'].fillna(-1)

In [37]:
df = df.drop(df[df['patient_leeftijd'].isnull()].index)

In [38]:
df[df.isnull().any(axis=1)].head(100)

,afspraak_id,afspraak_beginuur,afspraak_leadTime,afspraak_duur,afspraak_noShow,afspraak_aantalHerplanningen,afspraak_beschrijving,afspraak_specialisme,afspraak_arts,patient_aantalEerdereAfspraken,patient_dagenSindsVorigeAfspraak,patient_aantalAnnulaties,patient_aantalLaattijdigAnnulaties,patient_aantalAfsprakenDieDag,patient_stiptheid,patient_aantalNoShows,afspraak_weekdag,afspraak_maand,afspraak_isSchoolvakantie,patient_leeftijd,patient_geslacht,patient_dubieuzeBetaler,patient_burgerlijkeStaat,patient_nieuw,afstand


#### Nuttige kolommen toevoegen

In [39]:
df['patient_noShow_percentage'] = df['patient_aantalNoShows'] / df['patient_aantalEerdereAfspraken'] * 100
df['patient_laattijdigeAnnulatie_percentage'] = df['patient_aantalLaattijdigAnnulaties'] / df['patient_aantalEerdereAfspraken'] * 100

df['patient_noShow_percentage'] = df['patient_noShow_percentage'].fillna(0)
df['patient_laattijdigeAnnulatie_percentage'] = df['patient_laattijdigeAnnulatie_percentage'].fillna(0)

df['patient_noShow_percentage'] = df['patient_noShow_percentage'].round(2)
df['patient_laattijdigeAnnulatie_percentage'] = df['patient_laattijdigeAnnulatie_percentage'].round(2)


In [40]:
df['afspraak_isWeekend'] = df['afspraak_weekdag']>5
df['afspraak_isWeekend'] = df['afspraak_isWeekend'].astype(int)

In [41]:
df.to_csv('../data/afspraken_cleaned.csv', sep=';', index=False)